# Demand Profile Metrics Explorer

This notebook loads the demand profile CSV files alongside the dwelling summary data 
to compute the reporting metrics. Adjust the paths below if your repository layout differs.

In [ ]:
from pathlib import Path
import sys

REPO_ROOT = Path('..').resolve()
DEMAND_PROFILES_ROOT = REPO_ROOT / 'Codes' / 'Output' / 'DemandProfiles'
SUMMARY_FILE = REPO_ROOT / 'Codes' / 'Data' / '1R1C1P1S_filtered_filtered.csv'
OUTPUT_FILE = REPO_ROOT / 'Codes' / 'Output' / 'demand_metrics_summary.csv'

if str(REPO_ROOT) not in sys.path:
    sys.path.append(str(REPO_ROOT))

REPO_ROOT, DEMAND_PROFILES_ROOT, SUMMARY_FILE, OUTPUT_FILE

In [ ]:
import pandas as pd

from Codes.sourcecode.generate_demand_metrics import compute_metrics

metrics = compute_metrics(
    demand_profiles_root=DEMAND_PROFILES_ROOT,
    summary_path=SUMMARY_FILE,
)
metrics.head()

In [ ]:
# Save to CSV if desired
metrics.to_csv(OUTPUT_FILE, index=False)
print(f'Saved {len(metrics)} rows to {OUTPUT_FILE}')

## Peak demand reduction analysis

The cells below compute the reduction in peak electricity consumption when moving from the flat to the time-of-use tariff under extreme weather conditions and generate the requested visualisations.

In [ ]:
from Codes.sourcecode.analyze_peak_reduction import (
    HP_PROPORTION_TOU_COL,
    compute_peak_reduction,
    plot_histograms,
    plot_scatter_matrix,
)

peak_reduction = compute_peak_reduction(metrics)
peak_reduction.to_csv(
    DEMAND_PROFILES_ROOT / 'peak_demand_reduction_extreme.csv', index=False
)

hist_paths = plot_histograms(
    peak_reduction,
    DEMAND_PROFILES_ROOT / 'figures',
)

scatter_path = plot_scatter_matrix(
    peak_reduction,
    DEMAND_PROFILES_ROOT / 'figures' / 'peak_demand_reduction_scatter.png',
)

hhp_only = peak_reduction[
    peak_reduction['Technology'].astype(str).str.upper() == 'HHP'
]

if not hhp_only.empty:
    hp_hist_paths = plot_histograms(
        hhp_only,
        DEMAND_PROFILES_ROOT / 'figures',
        metric_col=HP_PROPORTION_TOU_COL,
        metric_label='Heat pump heat proportion (Time-of-use)',
        file_prefix='hp_heat_proportion_hist',
        units=None,
    )
    hp_scatter_path = plot_scatter_matrix(
        hhp_only,
        DEMAND_PROFILES_ROOT / 'figures' / 'hp_heat_proportion_scatter.png',
        metric_col=HP_PROPORTION_TOU_COL,
        metric_label='Heat pump heat proportion (Time-of-use)',
    )
else:
    print('No HHP rows available for heat proportion plots; skipping figure generation.')
    hp_hist_paths = {}
    hp_scatter_path = None

print('Saved peak reduction histogram(s):')
for tech, path in hist_paths.items():
    print(f' - {tech}: {path}')
print(f'Peak reduction scatter matrix: {scatter_path}')

if hp_hist_paths:
    print('Saved heat proportion histogram(s):')
    for tech, path in hp_hist_paths.items():
        print(f' - {tech}: {path}')
else:
    print('No heat proportion histograms generated.')
if hp_scatter_path:
    print(f'Heat proportion scatter matrix: {hp_scatter_path}')
else:
    print('Heat proportion scatter matrix not generated.')

peak_reduction.head()

